In [211]:
import pandas as pd

In [212]:
import pandas as pd
import pickle
import sklearn
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
import re
import os
from tqdm import tqdm
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()
import argparse
from nltk import word_tokenize, pos_tag
from collections import Counter

In [213]:
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/sujit/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /home/sujit/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/sujit/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/sujit/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/sujit/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [214]:
import re
def striphtml(data):
    p = re.compile(r'<(.*)>.*?|<(.*) />')
    return p.sub('', data)

# Function for Text cleaning

In [215]:
def preprocess(sentence):
    #print("setence before parsing",sentence)
    sentence=str(sentence)
    sentence = sentence.lower()
    sentence =  striphtml(sentence)
#     sentence = re.findall(r'http\S+', sentence)
#     sentence = re.sub(r'http\S+', '', sentence, flags=re.MULTILINE)
    sentence=sentence.replace('{html}',"")
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)
    filtered_words = [w for w in tokens if len(w) > 1 ]
#     filtered_words = [w for w in filtered_words if w not in stopwords.words('english')]
    #stem_words=[stemmer.stem(w) for w in filtered_words]
#     lemma_words=[lemmatizer.lemmatize(w) for w in filtered_words]
    #print("filtered word"," ".join(lemma_words))
    return " ".join(filtered_words)

# Load Dataset

In [216]:
train=pd.read_csv("train.csv")

In [217]:
test=pd.read_csv("test.csv")

In [218]:
dev=pd.read_csv("dev.csv")

# Check the Basic Distribution of Dtatasets

In [219]:
len(train)

2499

In [220]:
train.columns

Index(['text', 'class'], dtype='object')

In [221]:
train["class"].value_counts()

human    1253
bot      1246
Name: class, dtype: int64

In [222]:
len(test)

249

In [223]:
test["class"].value_counts()

bot      126
human    123
Name: class, dtype: int64

In [224]:
len(dev)

249

In [225]:
dev["class"].value_counts()

bot      131
human    118
Name: class, dtype: int64

# Change the type to 0 and 1: Binary
# Assign  Human generated text to class   0 label
# Assign  Human generated text to class   1 label

In [226]:
train.loc[ train['class'] == 'human', 'class'] = 0
train.loc[ train['class'] == 'bot', 'class'] = 1
dev.loc[ dev['class'] == 'human', 'class'] = 0
dev.loc[ dev['class'] == 'bot', 'class'] = 1
test.loc[ test['class'] == 'human', 'class'] = 0
test.loc[ test['class'] == 'bot', 'class'] = 1

In [227]:
test["class"].value_counts()

1    126
0    123
Name: class, dtype: int64

In [228]:
dev["class"].value_counts()

1    131
0    118
Name: class, dtype: int64

# Save updated Binary class Datatsets for further computations

In [229]:
train.to_csv("train_bin.csv", index=False)

In [230]:
test.to_csv("test_bin.csv", index=False)

In [231]:
dev.to_csv("dev_bin.csv", index=False)

# Apply preprocessing over datatsets

In [232]:
train['text'][0]

'YEA now that note GOOD'

# Preprocessing over training datatsets

In [233]:
for i in range(len(train)):
    train['text'][i] = preprocess (train['text'][i])

In [234]:
train['text'][1]

'listen to this charming man by the smiths'

# Preprocessing over test datatset

In [235]:
for i in range(len(test)):
    test['text'][i] = preprocess (test['text'][i])

In [236]:
test['text'][1]

'thank you pmbhutan for your gracious prayers and wishes from the people of bhutan we will work together to take this relationship to greater heights of peace and prosperity'

# # Preprocessing over test datatset

In [237]:
for i in range(len(dev)):
    dev['text'][i] = preprocess (dev['text'][i])

In [238]:
dev['text'][1]

'india has millennia old relations with oman we look forward to working hand in hand with hm sayyid haitham to further strengthen our strategic partnership'

# Save preprocessed  Datatsets for further computations

In [239]:
train.to_csv("train_bin_cleaned.csv", index=False)

In [240]:
test.to_csv("test_bin_cleaned.csv", index=False)

In [241]:
dev.to_csv("dev_bin_cleaned.csv", index=False)

# Exeract Part of Speech Tagging based feature 

# Function to exteract pos_tagging feature

In [242]:
def pos_feature(text):
    # Tokenize the text
    words = word_tokenize(text)

    # Perform POS tagging
    pos_tags = pos_tag(words)

    # Define a set of the 12 major POS tags
    major_pos_tags = {
        'N': 'Noun',
        'V': 'Verb',
        'R': 'Adverb',
        'J': 'Adjective',
        'P': 'Pronoun',
        'D': 'Determiner',
        'C': 'Conjunction',
        'X': 'Preposition',
        'U': 'Interjection',
        'M': 'Number',
        'S': 'Punctuation',
        'F': 'Foreign'
    }

    # Initialize a counter for each major POS tag
    pos_counts = Counter({tag: 0 for tag in major_pos_tags.values()})

    # Count the occurrences of each major POS tag
    for _, tag in pos_tags:
        major_tag = tag[0]
        if major_tag in major_pos_tags:
            pos_counts[major_pos_tags[major_tag]] += 1

    # Print the counts of major POS tags
#     print("Major POS Tag Counts:")
    feature_list=[]
    for tag, count in pos_counts.items():
#         print(f"{tag}: {count}")
        feature_list.append(count)
    return feature_list
        

# Exetract pos_tagging feature seperately for human generated and bot generated text for comparative study

In [244]:
train_pos_feature=[]  # exteract pos_tagging features for training datatsets
for i in range(len(train)):
    train_pos_feature.append(pos_feature(train['text'][i]))

In [245]:
# seperate training and testing features
human = train[train['class'] == 0]
bot = train[train['class'] == 1]

In [246]:
train_human_gen_text_feature=[] # exteract pos_tagging features for human generated text of training datatsets
for text in human["text"]:
    train_human_gen_text_feature.append(pos_feature(text))

In [248]:
len(train_human_gen_text_feature)

1253

In [249]:
train_human_gen_text_feature

[[2, 1, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0],
 [3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [46, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [11, 7, 1, 1, 1, 1, 2, 0, 0, 1, 0, 0],
 [10, 5, 2, 4, 1, 4, 1, 0, 0, 0, 0, 0],
 [7, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [9, 9, 2, 3, 1, 2, 0, 0, 0, 1, 0, 0],
 [4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [2, 2, 1, 3, 1, 1, 1, 0, 0, 0, 0, 0],
 [9, 2, 1, 2, 2, 2, 1, 0, 0, 0, 0, 0],
 [5, 4, 1, 0, 4, 1, 0, 0, 0, 1, 0, 0],
 [1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [3, 1, 1, 2, 0, 1, 0, 0, 0, 0, 0, 0],
 [4, 5, 1, 0, 3, 1, 1, 0, 0, 1, 0, 0],
 [8, 4, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
 [1, 2, 0, 2, 0, 1, 0, 0, 0, 0, 0, 0],
 [4, 7, 3, 2, 2, 0, 0, 0, 0, 0, 0, 0],
 [4, 0, 0, 2, 0, 1, 1, 0, 0, 0, 0, 0],
 [9, 8, 1, 2, 2, 1, 2, 0, 0, 0, 0, 0],
 [4, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0],
 [2, 3, 3, 0, 2, 0, 0, 0, 0, 0, 0, 0],
 [3, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
 [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [6, 2, 1, 2, 0, 3, 0, 0, 0, 0, 0, 0],
 [6, 2, 1, 3, 2, 1, 0, 0, 0, 1, 0, 0],
 [2, 0, 0, 0, 0, 0, 0,

In [250]:
#convert list of list to numpy array for count of human generated text pos tagging feature feature
import numpy as np
u= np.array(train_human_gen_text_feature)

In [251]:
train_bot_gen_text_feature=[] #exteract pos_tagging features for bot generated text of training datatsets
for text in bot["text"]:
    train_bot_gen_text_feature.append(pos_feature(text))

In [252]:
#convert list of list to numpy array for count of boat feature
import numpy as np
x= np.array(train_bot_gen_text_feature)

In [253]:
x

array([[2, 0, 2, ..., 0, 0, 0],
       [2, 2, 0, ..., 2, 0, 0],
       [2, 0, 1, ..., 0, 0, 0],
       ...,
       [8, 3, 0, ..., 0, 0, 0],
       [2, 0, 0, ..., 0, 0, 0],
       [2, 2, 1, ..., 0, 0, 0]])

# compuare the different post tagging count in human generated text and bot generated text

In [254]:
# count the total number of time pos_tag appeared in boat generated text
# number of nonzero pos count in each row for boat generated text
result= x.sum(axis=0)
result

array([6497, 3758, 1031, 1824, 1294, 2715,  751,    0,    9,  257,    0,
          7])

In [255]:
len(x)

1246

In [256]:
# count the total number of time pos_tag appeared in human generated text
# number of nonzero pos count in each row for human generated text
result= u.sum(axis=0)
result

array([6517, 4068, 1409, 2136, 1479, 1524,  745,    0,    3,  271,    0,
         10])

In [257]:
len(u)

1253

# count the number of pos tagging  appeared in each bot generated text.

In [260]:
non_zero_bot=[]

In [261]:
# number of nonzero pos count in each row for boat generated text
for i in range(len(x)):
    
    k= np.count_nonzero(x[i,:])
    non_zero_bot.append(k)

In [262]:
len(non_zero_bot)

1246

In [263]:
df=pd.DataFrame(non_zero_bot)
df.describe()

,0
count,1246.000000
mean,4.865169
std,1.739073
min,0.000000
25%,4.000000
50%,5.000000
75%,6.000000
max,8.000000


# count the number of pos tagging  appeared in each human generated text.

In [265]:
# number of nonzero pos count in each row for human generated text
non_zero_human=[]
for i in range(len(u)):
    
    q= np.count_nonzero(u[i,:])
    non_zero_human.append(k)

In [266]:
df1= pd.DataFrame(non_zero_human)

In [267]:
df1.describe()

,0
count,1253.0
mean,4.0
std,0.0
min,4.0
25%,4.0
50%,4.0
75%,4.0
max,4.0


# Function to check the spelling error in a text doccument and return number of words with mistakes spelling

In [269]:
import enchant

# Create a dictionary object for your preferred language (e.g., English)
# You can specify the language using an appropriate language code.
# For English, 'en_US' is used.
dictionary = enchant.Dict("en_US")

# Sample text with potential spelling errors
# text = "This is an exmaple of a sentence with speling errors."

# Split the text into words
words = text.split()

# Check each word for spelling errors
print(len(train))
train_spelling_error=[]
for i in range(len(train)):
    spelling_errors = []
    
    for word in train['text'][i].split():
        if not dictionary.check(word):
            spelling_errors.append(word)
    train_spelling_error.append([len(spelling_errors)])
    
test_spelling_error=[]
for i in range(len(test)):
    spelling_errors = []
    
    for word in test['text'][i].split():
        if not dictionary.check(word):
            spelling_errors.append(word)
    test_spelling_error.append([len(spelling_errors)])
    

# # Print the list of spelling errors
# print("Spelling Errors:", spelling_errors)

2499


In [270]:
len(train_spelling_error)

2499

In [271]:
train_spelling_error

[[0],
 [0],
 [0],
 [0],
 [11],
 [1],
 [1],
 [1],
 [2],
 [2],
 [1],
 [3],
 [1],
 [2],
 [0],
 [3],
 [4],
 [1],
 [1],
 [1],
 [0],
 [0],
 [0],
 [0],
 [3],
 [0],
 [0],
 [1],
 [4],
 [0],
 [2],
 [0],
 [5],
 [0],
 [1],
 [1],
 [1],
 [1],
 [1],
 [2],
 [0],
 [2],
 [6],
 [2],
 [2],
 [4],
 [1],
 [2],
 [2],
 [1],
 [0],
 [2],
 [3],
 [1],
 [2],
 [1],
 [2],
 [1],
 [7],
 [0],
 [0],
 [2],
 [1],
 [1],
 [1],
 [0],
 [1],
 [0],
 [3],
 [0],
 [5],
 [0],
 [3],
 [2],
 [2],
 [8],
 [2],
 [4],
 [0],
 [1],
 [2],
 [4],
 [2],
 [0],
 [4],
 [5],
 [2],
 [3],
 [1],
 [7],
 [0],
 [0],
 [0],
 [1],
 [1],
 [5],
 [2],
 [2],
 [2],
 [3],
 [1],
 [0],
 [3],
 [0],
 [0],
 [1],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [3],
 [6],
 [1],
 [0],
 [19],
 [2],
 [1],
 [0],
 [1],
 [1],
 [8],
 [0],
 [0],
 [1],
 [1],
 [3],
 [5],
 [4],
 [0],
 [6],
 [1],
 [0],
 [0],
 [2],
 [2],
 [0],
 [3],
 [3],
 [4],
 [2],
 [3],
 [6],
 [0],
 [3],
 [3],
 [0],
 [4],
 [2],
 [2],
 [0],
 [1],
 [1],
 [2],
 [1],
 [3],
 [1],
 [2],
 [2],
 [2],
 [4],
 [1],
 [3],
 [1],
 [1],
 [

In [272]:
test_spelling_error

[[2],
 [2],
 [0],
 [2],
 [0],
 [0],
 [1],
 [0],
 [1],
 [0],
 [1],
 [0],
 [1],
 [0],
 [1],
 [4],
 [1],
 [1],
 [0],
 [5],
 [0],
 [0],
 [7],
 [1],
 [2],
 [0],
 [0],
 [0],
 [1],
 [0],
 [1],
 [4],
 [2],
 [0],
 [4],
 [2],
 [0],
 [0],
 [0],
 [2],
 [1],
 [5],
 [0],
 [1],
 [1],
 [0],
 [2],
 [1],
 [1],
 [3],
 [6],
 [1],
 [1],
 [0],
 [1],
 [1],
 [0],
 [0],
 [1],
 [3],
 [5],
 [0],
 [1],
 [3],
 [2],
 [1],
 [3],
 [1],
 [1],
 [0],
 [2],
 [5],
 [1],
 [0],
 [0],
 [1],
 [2],
 [1],
 [0],
 [0],
 [2],
 [2],
 [0],
 [0],
 [0],
 [0],
 [1],
 [1],
 [1],
 [0],
 [0],
 [4],
 [4],
 [0],
 [2],
 [2],
 [1],
 [2],
 [1],
 [1],
 [2],
 [0],
 [2],
 [2],
 [0],
 [1],
 [2],
 [3],
 [0],
 [1],
 [0],
 [0],
 [0],
 [1],
 [0],
 [1],
 [0],
 [4],
 [3],
 [0],
 [4],
 [4],
 [1],
 [5],
 [5],
 [2],
 [1],
 [2],
 [0],
 [0],
 [2],
 [0],
 [1],
 [0],
 [0],
 [1],
 [1],
 [2],
 [1],
 [4],
 [0],
 [1],
 [3],
 [3],
 [2],
 [2],
 [1],
 [1],
 [4],
 [5],
 [4],
 [6],
 [9],
 [2],
 [1],
 [0],
 [1],
 [3],
 [0],
 [0],
 [3],
 [0],
 [3],
 [3],
 [1],
 [1],
 [0]

In [89]:
!pip install textstat

  Using cached textstat-0.7.3-py3-none-any.whl (105 kB)
  Using cached pyphen-0.14.0-py3-none-any.whl (2.0 MB)


# Eastimate the different Readability score of text

In [273]:
from textstat import flesch_kincaid_grade
from textstat import gunning_fog
from textstat import smog_index
from textstat import dale_chall_readability_score

# Sample text
train_readability_features = []
for i in range(len(train)):
    #temp = []
    # Calculate the Flesch-Kincaid Grade Level
    grade_level = flesch_kincaid_grade(train['text'][i])
    grade_level1 = gunning_fog(train['text'][i])
    grade_level2 = smog_index(train['text'][i])
    grade_level2 = smog_index(train['text'][i])
    grade_level3 = dale_chall_readability_score(train['text'][i])
    temp = [grade_level, grade_level1, grade_level2, grade_level3]
    train_readability_features.append(temp)
    
train_readability_features = np.array(train_readability_features)


#print(train_readability_features)
print('------------------------------------------------')
test_readability_features = []
for i in range(len(test)):
    #temp = []
    # Calculate the Flesch-Kincaid Grade Level
    grade_level = flesch_kincaid_grade(test['text'][i])
    grade_level1 = gunning_fog(test['text'][i])
    grade_level2 = smog_index(test['text'][i])
    grade_level2 = smog_index(test['text'][i])
    grade_level3 = dale_chall_readability_score(test['text'][i])
    temp = [grade_level, grade_level1, grade_level2, grade_level3]
    test_readability_features.append(temp)


#print(test_readability_features)
test_readability_features = np.array(test_readability_features)

# print("Flesch-Kincaid Grade Level:", grade_level)
# print("Flesch-Kincaid Grade Level:", grade_level1 )
# print("Flesch-Kincaid Grade Level:", grade_level2 )
# print("Flesch-Kincaid Grade Level:", grade_level3 )

------------------------------------------------


# Exteract the post tagging features for training datatset

In [101]:
train_pos_features=[]
for i in range(len(train)):
    train_pos_features.append(pos_feature(train['text'][i]))
train_pos_features = np.array(train_pos_features)

# Exteract the post tagging features for text datatset

In [275]:
test_pos_features=[]
for i in range(len(test)):
    test_pos_features.append(pos_feature(test['text'][i]))
test_pos_features = np.array(test_pos_features)

In [276]:
train_pos_features

array([[2, 0, 2, ..., 0, 0, 0],
       [2, 1, 1, ..., 0, 0, 0],
       [2, 2, 0, ..., 2, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [2, 2, 1, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0]])

In [277]:
test_pos_features

array([[2, 2, 1, ..., 0, 0, 0],
       [9, 3, 1, ..., 1, 0, 0],
       [3, 5, 1, ..., 1, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [3, 4, 1, ..., 2, 0, 0],
       [7, 3, 0, ..., 0, 0, 0]])

# Exteract features based number of different Post_tagg appeared in text .

In [285]:
train_u= np.array(train_pos_features)
test_v= np.array(test_pos_features)

In [292]:
train_total_pos_count=[]
for i in range(len(train_u)):
    
    k= np.count_nonzero(train_u[i,:])
    train_total_pos_count.append([k])

In [293]:
train_total_pos_count

[[2],
 [4],
 [5],
 [4],
 [2],
 [1],
 [6],
 [5],
 [2],
 [8],
 [6],
 [7],
 [3],
 [8],
 [8],
 [7],
 [1],
 [8],
 [7],
 [7],
 [6],
 [7],
 [5],
 [2],
 [7],
 [5],
 [2],
 [1],
 [5],
 [7],
 [5],
 [4],
 [5],
 [3],
 [4],
 [7],
 [6],
 [6],
 [3],
 [7],
 [4],
 [5],
 [5],
 [7],
 [4],
 [7],
 [3],
 [4],
 [4],
 [3],
 [1],
 [5],
 [6],
 [5],
 [7],
 [1],
 [4],
 [1],
 [6],
 [5],
 [5],
 [5],
 [6],
 [6],
 [5],
 [2],
 [4],
 [5],
 [4],
 [5],
 [6],
 [3],
 [6],
 [4],
 [7],
 [7],
 [3],
 [7],
 [4],
 [3],
 [1],
 [7],
 [6],
 [5],
 [3],
 [5],
 [1],
 [6],
 [3],
 [7],
 [2],
 [6],
 [5],
 [3],
 [3],
 [6],
 [6],
 [4],
 [8],
 [7],
 [5],
 [5],
 [2],
 [3],
 [5],
 [6],
 [3],
 [6],
 [2],
 [3],
 [2],
 [7],
 [4],
 [7],
 [3],
 [7],
 [5],
 [3],
 [3],
 [5],
 [2],
 [2],
 [6],
 [2],
 [7],
 [1],
 [5],
 [2],
 [6],
 [5],
 [5],
 [4],
 [5],
 [1],
 [7],
 [7],
 [6],
 [7],
 [7],
 [7],
 [6],
 [4],
 [7],
 [7],
 [5],
 [6],
 [6],
 [5],
 [7],
 [5],
 [7],
 [3],
 [4],
 [4],
 [5],
 [5],
 [6],
 [3],
 [8],
 [7],
 [5],
 [8],
 [6],
 [6],
 [3],
 [1],
 [1]

In [294]:
test_total_pos_count=[]
for i in range(len(test_v)):
    
    k= np.count_nonzero(test_v[i,:])
    test_total_pos_count.append([k])

In [295]:
test_total_pos_count

[[6],
 [8],
 [7],
 [4],
 [8],
 [5],
 [6],
 [4],
 [5],
 [3],
 [7],
 [6],
 [6],
 [7],
 [3],
 [6],
 [4],
 [2],
 [6],
 [5],
 [4],
 [6],
 [6],
 [4],
 [7],
 [1],
 [3],
 [3],
 [4],
 [6],
 [6],
 [5],
 [4],
 [2],
 [7],
 [4],
 [4],
 [2],
 [3],
 [4],
 [3],
 [7],
 [2],
 [4],
 [5],
 [5],
 [4],
 [5],
 [6],
 [7],
 [7],
 [1],
 [3],
 [1],
 [1],
 [5],
 [6],
 [5],
 [7],
 [6],
 [4],
 [5],
 [7],
 [4],
 [6],
 [5],
 [6],
 [8],
 [4],
 [3],
 [7],
 [6],
 [8],
 [1],
 [4],
 [5],
 [2],
 [6],
 [0],
 [4],
 [4],
 [3],
 [3],
 [2],
 [7],
 [1],
 [6],
 [4],
 [1],
 [3],
 [6],
 [5],
 [5],
 [6],
 [4],
 [5],
 [7],
 [5],
 [1],
 [5],
 [7],
 [4],
 [7],
 [7],
 [3],
 [1],
 [2],
 [4],
 [4],
 [1],
 [4],
 [1],
 [2],
 [3],
 [3],
 [3],
 [1],
 [4],
 [6],
 [5],
 [7],
 [6],
 [3],
 [4],
 [6],
 [6],
 [3],
 [5],
 [5],
 [4],
 [3],
 [5],
 [1],
 [3],
 [5],
 [6],
 [5],
 [5],
 [7],
 [5],
 [1],
 [6],
 [4],
 [4],
 [5],
 [3],
 [8],
 [5],
 [6],
 [7],
 [7],
 [6],
 [7],
 [2],
 [1],
 [1],
 [3],
 [7],
 [5],
 [3],
 [4],
 [7],
 [7],
 [7],
 [6],
 [2],
 [4]

# Concantenate all the features and form a feature vector fot training and testing

In [303]:
train_features = np.concatenate((train_readability_features, train_pos_features,train_total_pos_count,train_spelling_error), axis=1)

In [304]:
train_features.shape

(2499, 18)

In [305]:
test_features = np.concatenate((test_readability_features, test_pos_features,test_total_pos_count, test_spelling_error), axis=1)

In [306]:
test_features.shape

(249, 18)

# Dump all the exteracted feature over training and text datatsets for future experiments

In [309]:
df= pd.DataFrame(train_features)
df.to_csv("train_features.csv",index=None)

In [310]:
df1= pd.DataFrame(test_features)
df1.to_csv("test_features.csv",index=None)

In [313]:
y_train= np.array(train["class"].tolist())

In [314]:
y_train

array([1, 0, 1, ..., 0, 1, 0])

In [315]:
y_test= np.array(test["class"].tolist())

In [316]:
y_test

array([0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1,
       1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1,
       0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1,
       0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0,
       1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0,
       1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0,
       1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1,
       1, 0, 0, 0, 0, 0, 1])

# Train a Support vector machine (SVM) classifier with exteracted features

In [318]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC



# Model Evaluation fucntion with accuracy, f-measure and class wise f1 score

In [325]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

def my_score(y_test, y_pred):
    acc =accuracy_score(y_test, y_pred)
    f1= f1_score(y_test, y_pred, average='macro')
    
    f1_classwise = f1_score(y_test, y_pred, average=None)
    
    
    return acc,f1,f1_classwise


# Fit SVM

In [323]:
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(train_features, y_train)
y_pred = clf.predict(test_features)

In [327]:
score= my_score(y_test, y_pred)
print(score)




(0.6706827309236948, 0.6704222623966942, array([0.66115702, 0.6796875 ]))


# Fit Ensemble Model Adaboost classifier with exteracted features

In [348]:
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier(n_estimators=140, random_state=0)

In [349]:
clf.fit(train_features, y_train)

AdaBoostClassifier(n_estimators=140, random_state=0)

In [350]:
y_pred = clf.predict(test_features)

In [351]:
score= my_score(y_test, y_pred)
print(score)

(0.6746987951807228, 0.6746148261676213, array([0.66938776, 0.6798419 ]))


# verify confucion matrix to understand predictions

In [352]:
from sklearn.metrics import confusion_matrix

In [353]:
confusion_matrix(y_test, y_pred)

array([[82, 41],
       [40, 86]])

# Fit Ensemble Model Gradient Boosting  classifier with exteracted features

In [354]:
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier

In [355]:
clf = GradientBoostingClassifier(n_estimators=150, learning_rate=1.0,
    max_depth=1, random_state=0).fit(train_features, y_train)


In [356]:
y_pred = clf.predict(test_features)

In [357]:
score= my_score(y_test, y_pred)
print(score)

(0.6586345381526104, 0.6585464225215778, array([0.66403162, 0.65306122]))


In [358]:
confusion_matrix(y_test, y_pred)

array([[84, 39],
       [46, 80]])